In [11]:
! pip install tensorflow autokeras
! pip install python-telegram-bot==13.3

In [12]:
import logging
from telegram import ReplyKeyboardMarkup, Update
from telegram.ext import (Updater, CommandHandler, MessageHandler, filters,
                          ConversationHandler, CallbackQueryHandler, CallbackContext)
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import joblib
import json

import autokeras as ak

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Загрузка модели
model = load_model('/content/drive/MyDrive/NMIC/autokeras_structured_data_model')

# Загрузка label_encoder
label_encoder = joblib.load('/content/drive/MyDrive/NMIC/label_encoder.joblib')

# Столбцы X_train из JSON файла в список
with open('/content/drive/MyDrive/NMIC/columns.json', 'r') as json_file:
    train_columns = json.load(json_file)

In [20]:
data = {
    'Раса': ['Азиатская', 'Европейская', 'Другая'],
    'Пол': ['Мужской', 'Женский'],
    'Возраст': ['До 70', '>70'],
    'Статус курения': ['В настоящее время', 'Курение в прошлом (бросил более 1 месяца до 1 года)'],
    'ECOG': ['0-1', '2'],
    'Есть опухолевая нагрузка? (симптомная опухоль)': ['да', 'нет'],
    'Ко-мутации KRAS': ['да', 'нет'],
    'Ко-мутации p53.': ['да', 'нет'],
    'Ко-мутации STK11': ['да', 'нет'],
    'Ко-мутации KEAP1': ['да', 'нет'],
    'Срок от окончания ХЛТ': ['до 42 дней', 'от 43 до 60 дней', 'более 61 дня'],
    'Молекулярный статус (только для неплоскоклеточного рака)': ['нет мутаций', 'не исследовались', 'EGFR редкий вариант'],
    'PD-L1 статус': ['Не исследовался', 'Менее 1%', 'Более 1%'],
    'Предпочтение пациента по ответу на терапию': ['Результативность лечения', 'сохранение качества жизни'],
}

# Названия столбцов для вопросов
columns = data.keys()

# Словарь для хранения ответов пользователя
user_answers = {}

# Функции для ConversationHandler
def ask_next_column(update: Update, context: CallbackContext) -> None:
    current_column = list(data.keys())[len(user_answers)]
    reply_keyboard = [data[current_column]]
    update.message.reply_text(
        f'Выберите значение для столбца "{current_column}":',
        reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)
    )
    return 'answer'

def receive_answer(update: Update, context: CallbackContext) -> None:
    current_column = list(data.keys())[len(user_answers)]
    user_answers[current_column] = update.message.text
    if len(user_answers) < len(data):
        return ask_next_column(update, context)
    else:
        return end_dialog(update, context)
    return 'answer'

def end_dialog(update: Update, context: CallbackContext) -> None:
    df = pd.DataFrame([user_answers])
    df_encoded = pd.get_dummies(df)

    # Проверяем и добавляем недостающие столбцы
    missing_columns = [col for col in train_columns if col not in df_encoded.columns]
    df_encoded = pd.concat([df_encoded, pd.DataFrame(columns=missing_columns)], axis=1)
    df_encoded[missing_columns] = 0
    df_encoded = df_encoded[train_columns]

    prediction = model.predict(df_encoded)
    predicted_label = label_encoder.inverse_transform(prediction.argmax(axis=1))

    # Форматированный вывод с использованием f-строк
    formatted_response = f"""
Ответ эксперта: {predicted_label[0].split('_')[0]}
Уверенность: {predicted_label[0].split('_')[1]}
Комментарий: {predicted_label[0].split('_')[2]}
Альтернатива, если 50% - обязательно: {predicted_label[0].split('_')[3]}
"""
    update.message.reply_text(formatted_response)
    return ConversationHandler.END

def start_dialog(update: Update, context: CallbackContext) -> None:
    update.message.reply_text("Привет! Я бот для предсказания лечения больных раком легких. Введи данные о пациенте, выбирая нужный ответ на каждый вопрос.")

    return ask_next_column(update, context)

# Настройка и запуск ConversationHandler
conv_handler = ConversationHandler(
    entry_points=[CommandHandler('start', start_dialog)],
    states={
        'answer': [MessageHandler(filters.Filters.text & ~filters.Filters.command, receive_answer)],
    },
    fallbacks=[CommandHandler('end', end_dialog)]
)

# Настройка и запуск бота
def main() -> None:
    updater = Updater("6707634379:AAHNph92ymAQ_vsb5aHLSP6heVMs6jlkEnY")

    dp = updater.dispatcher
    dp.add_handler(conv_handler)

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

1/1 [==============================] - 0s 23ms/step
